In [1]:
from tyssue.generation import hexa_grid3d

In [12]:
import ipyvolume as ipv
from tyssue.draw import sheet_view, highlight_cells

from tyssue import Epithelium, BulkGeometry
from tyssue.generation import from_3d_voronoi, hexa_grid3d
from scipy.spatial import Voronoi

cells = hexa_grid3d(3, 3, 3)
datasets = from_3d_voronoi(Voronoi(cells))
bulk = Epithelium('bulk', datasets)
bulk.reset_topo()
bulk.reset_index(order=True)
bulk.sanitize()

BulkGeometry.update_all(bulk)

# We'll see next how to configure visualisation

bulk.face_df['visible'] = False

ipv.clear()
fig2, mesh = sheet_view(bulk, mode="3D",
                        face={"visible":False})
fig2

Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…

In [6]:
bulk.get_opposite_faces()

In [9]:
import numpy as np

In [10]:
np.unique(bulk.face_df["opposite"])


array([-1])

In [134]:
bulk.face_df.take([12, 13, 12], axis=0)

,x,y,z,num_sides,area,perimeter,is_alive,id,opposite,visible
face,,,,,,,,,,
12,1.25,0.433013,0.866025,4,0.375,2.489276,1,0,-1,False
13,0.75,0.433013,0.866025,4,0.375,2.489276,1,0,-1,False
12,1.25,0.433013,0.866025,4,0.375,2.489276,1,0,-1,False


In [135]:
a = np.arange(120).reshape(12, 10)

In [140]:
a.take([8, 5, 8], axis=0)

array([[80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89]])

In [108]:
def from_3d_voronoi(voro):
    specs3d = bulk_spec()

    el_idx = []
    n_single_faces = len(voro.ridge_vertices)

    for f_idx, (rv, rp) in enumerate(zip(voro.ridge_vertices, voro.ridge_points)):

        if -1 in rv:
            continue
        face_verts = voro.vertices[rv]
        f_center = face_verts.mean(axis=0)
        c0 = voro.points[rp[0]]
        ctof = f_center - c0

        for rv0, rv1 in zip(rv, np.roll(rv, 1, axis=0)):
            fv0 = voro.vertices[rv0]
            fv1 = voro.vertices[rv1]
            edge_v = fv1 - fv0
            fto0 = fv0 - f_center
            normal = np.cross(fto0, edge_v)
            dotp = np.dot(ctof, normal)
            if np.sign(dotp) > 0:
                el_idx.append([rv0, rv1, f_idx, rp[0]])
                el_idx.append([rv1, rv0, f_idx+n_single_faces, rp[1]])
            else:
                el_idx.append([rv1, rv0, f_idx, rp[0]])
                el_idx.append([rv0, rv1, f_idx+n_single_faces, rp[1]])

    el_idx = np.array(el_idx)

    coords = ["x", "y", "z"]
    edge_idx = pd.Index(range(el_idx.shape[0]), name="edge")
    edge_df = make_df(edge_idx, specs3d["edge"])

    for i, elem in enumerate(["srce", "trgt", "face", "cell"]):
        edge_df[elem] = el_idx[:, i]

    vert_idx = pd.Index(range(voro.vertices.shape[0]), name="vert")
    vert_df = make_df(vert_idx, specs3d["vert"])
    vert_df[coords] = voro.vertices
    included_verts = edge_df["srce"].unique()
    vert_df = vert_df.loc[included_verts].copy()

    cell_idx = pd.Index(range(voro.points.shape[0]), name="cell")
    cell_df = make_df(cell_idx, specs3d["cell"])
    cell_df[coords] = voro.points
    included_cells = edge_df["cell"].unique()
    cell_df = cell_df.loc[included_cells].copy()

    included_faces = edge_df["face"].unique()
    face_df = make_df(included_faces, specs3d["face"])
    edge_df.sort_values(by="cell", inplace=True)

    datasets = {"vert": vert_df, "edge": edge_df, "face": face_df, "cell": cell_df}
    return datasets

In [109]:
cells = hexa_grid3d(3, 3, 3)
datasets = from_3d_voronoi(Voronoi(cells))
bulk = Epithelium('bulk', datasets)
bulk.reset_topo()
bulk.reset_index(order=True)
bulk.sanitize()

BulkGeometry.update_all(bulk)
bulk.get_opposite_faces()

assert np.unique(bulk.face_df["opposite"]).shape[0] > 1

In [111]:
bulk.validate()

True

In [129]:
bulk.edge_df["cell"].unique().size

4

In [133]:
bulk.edge_df.groupby("face").apply(lambda df: df['cell'].unique().size).max()

1

In [ ]:
assert bulk.groupby("face").bin

In [113]:
bulk.face_df


,x,y,z,num_sides,area,perimeter,is_alive,id,opposite,visible
face,,,,,,,,,,
0,0.041667,1.419319,0.866025,3,0.630208,4.067011,1,0,-1,False
1,-0.375000,1.876388,1.118616,4,0.953509,5.795600,1,0,-1,False
2,0.750000,1.299038,0.866025,4,0.375000,2.489276,1,0,32,False
3,1.000000,1.890822,0.866025,5,0.752604,3.571293,1,0,33,False
4,1.250000,0.649519,1.299038,4,0.272431,2.158484,1,0,-1,False
5,0.750000,0.649519,1.299038,4,0.272431,2.158484,1,0,-1,False
6,0.500000,0.914138,0.866025,6,0.437500,2.590033,1,0,-1,False
7,1.500000,0.914138,0.866025,6,0.437500,2.590033,1,0,34,False
8,1.250000,1.299038,0.866025,4,0.375000,2.489276,1,0,35,False


In [112]:
highlight_cells(bulk, [3,], reset_visible=True)


ipv.clear()
fig2, mesh = sheet_view(bulk, mode="3D",
                        face={"visible":True})
fig2

Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…